In [48]:
import pyvistaqt as pvqt # For updating plots real time

from AI.fenics import *
from AI.material_handler import silicon, rubber

[PyvistQT documentation](https://qtdocs.pyvista.org/api_reference.html)


For scripts use:

_from PyQt5.QtWidgets import QApplication_
_app = QApplication(sys.argv)_
_app.exec_()

TODO:
- Add database of materials
- Add more complicated meshes
- Show boundary conditions
- STIMULI handling

In [49]:
# Create a library of meshes

cuboid = MeshFromFile(
    'C:/Users/majag/Desktop/marble/MARBLE/AI/meshes/block.mesh'
)

elbow = MeshFromFile(
    'C:/Users/majag/Desktop/marble/MARBLE/AI/meshes/elbow.mesh'
)

perfusion = MeshFromFile(
    'C:/Users/majag/Desktop/marble/MARBLE/AI/meshes/perfusion_micro3d.mesh'
)

hexa_mesh = MeshFromFile(
        'C:/Users/majag/Desktop/marble/MARBLE/AI/meshes/acoustics_mesh3d.mesh'
)

hexa_flat = GridMesh(15, 15, z_function=flat)

hexa_wave = GridMesh(15, 15, z_function=wave)

hexa_concave = GridMesh(15, 15, z_function=concave)

hexa_convex = GridMesh(15, 15, z_function=convex)

sfepy: reading mesh (C:/Users/majag/Desktop/marble/MARBLE/AI/meshes/block.mesh)...
sfepy:   number of vertices: 525
sfepy:   number of cells:
sfepy:     3_8: 320
sfepy: ...done in 0.01 s
sfepy: reading mesh (C:/Users/majag/Desktop/marble/MARBLE/AI/meshes/elbow.mesh)...
sfepy:   number of vertices: 1823
sfepy:   number of cells:
sfepy:     3_4: 8161
sfepy: ...done in 0.06 s
sfepy: reading mesh (C:/Users/majag/Desktop/marble/MARBLE/AI/meshes/perfusion_micro3d.mesh)...
sfepy:   number of vertices: 3728
sfepy:   number of cells:
sfepy:     3_4: 18161
sfepy: ...done in 0.08 s
sfepy: reading mesh (C:/Users/majag/Desktop/marble/MARBLE/AI/meshes/acoustics_mesh3d.mesh)...
sfepy:   number of vertices: 5272
sfepy:   number of cells:
sfepy:     3_8: 4380
sfepy: ...done in 0.09 s
sfepy: reading mesh (meshes/mesh.vtk)...
sfepy:   number of vertices: 450
sfepy:   number of cells:
sfepy:     3_8: 196
sfepy: ...done in 0.03 s
sfepy: reading mesh (meshes/mesh.vtk)...
sfepy:   number of vertices: 450
sfe

In [50]:
def add_mesh(mesh, plotter):
    """
    Adds the mesh in the .vtk format to the plotter
    """

    plotter.add_mesh(
        mesh,
        color='deepskyblue',
        show_edges=True,
        smooth_shading=True,
        edge_color='wheat',
    )

In [51]:
mesh_boost = hexa_concave
_fenics = FENICS(mesh_boost, silicon)

U, IDs = None, None

In [52]:
def apply_volume_force(mesh, plotter, fenics):
    """
    Function that applies a volume (stable) force across the whole mesh
    :param fenics: SfePy solver
    :param plotter: PyVista plotter
    :param mesh: Mesh object including the Pyvista mesh
    """

    u = fenics.apply_volume_force()
    mesh.update(u)

    # Reloading the mesh to the scene
    plotter.clear()
    add_mesh(mesh.vtk_mesh, plotter)

    # Redrawing
    plotter.update()

In [53]:
def apply_cell_force(cell, fenics, plotter, mesh):
    """
    Function that applies a force to the vertex that was picked
    :param cell: cell object that was picked form the callback
    :param fenics: SfePy solver
    :param plotter: PyVista plotter
    :param mesh: Mesh object including the Pyvista mesh
    """
    global U, IDs

    vertex_ids = mesh.get_vertex_ids_from_coords(cell.points)

    u = fenics.apply_vertex_specific_force(vertex_ids, F=-20)
    mesh.update(u)

    U = u
    IDs = vertex_ids

    # Reloading the mesh to the scene
    plotter.clear()
    add_mesh(mesh.vtk_mesh, plotter)

    # Redrawing
    plotter.update()

In [54]:
"""
Background plotter

From documentation:
PyVista provides a plotter that enables users to create a rendering window in the background,
that remains interactive while the user performs their processing.

See documentation for more details:
https://qtdocs.pyvista.org/api_reference.html#pyvistaqt.BackgroundPlotter
"""

_plotter = pvqt.BackgroundPlotter()

# Add the mesh to the plotter
add_mesh(mesh_boost.vtk_mesh, _plotter)

# Add the event on the press of the space bar
_plotter.add_key_event("space", lambda: apply_volume_force(mesh_boost, _plotter, _fenics))


_plotter.enable_cell_picking(
    callback=lambda cell: apply_cell_force(cell, _fenics, _plotter, mesh_boost),
    font_size=10,
    color='black',
    point_size=30,
    style='wireframe',
    line_width=5,
    through=False
)

_plotter.showFullScreen()

In [55]:
print('maximum displacement x:', np.abs(U[:, 0]).max())
print('maximum displacement y:', np.abs(U[:, 1]).max())
print('maximum displacement z:', np.abs(U[:, 2]).max())

maximum displacement x: 0.0007775502190402611
maximum displacement y: 0.0007775502190402598
maximum displacement z: 0.039524792866405895


In [56]:
IDs

[112, 113, 127, 128]